In [27]:
import pandas as pd
from gurobipy import Model, GRB, quicksum
import time
from datetime import timedelta

In [28]:
# Loading the data
filename = "2025-04-09+DATEN+OR+PRAKTKUM.xlsx"
start_time = time.time()
df_terminals = pd.read_excel(filename, sheet_name="Standorte inkl. Kapa´s", nrows=40)
df_postalcodes = pd.read_excel(filename, sheet_name="PLZ inkl. Sendungsmengen", nrows=8185)
df_distance = pd.read_excel(filename, sheet_name="DISTANZEN PLZ-TERMINAL", nrows=319138)
df_postalcodes_neighbors = pd.read_excel(filename, sheet_name="ADJAZENZ (BENACHBARTE PLZ)", nrows=47837)

elapsed_time_secs = time.time() - start_time
s = timedelta(seconds=round(elapsed_time_secs))
print(s)

0:00:11


In [29]:
print("-------------------------------------------------------")
print("Distance sheet columns:", df_distance.columns.tolist())
print("\nTerminal sheet columns:", df_terminals.columns.tolist())
print("\nPostal code sheet columns:",df_postalcodes.columns.tolist())
print("-------------------------------------------------------")

-------------------------------------------------------
Distance sheet columns: ['von PLZ (TEXT)', 'von PLZ (ZAHL)', 'nach Terminalcode', 'nach Terminal', 'Distanz (km)', 'Fahrzeit (min)']

Terminal sheet columns: ['Terminal code', 'Terminal name', 'Country ISO code', 'X-DOCK ADDRESS: street and building number', 'X-DOCK ADDRESS: postal code', 'X-DOCK ADDRESS: city', 'LAT', 'LON', 'Partner', 'MAX. KAPA SE (# ZUSTELLUNGEN pro Tag)']

Postal code sheet columns: ['PLZ (TEXT)', 'PLZ (ZAHL)', 'Name', 'LAT', 'LON', 'ZUORDNUNG IST', 'PARTNER', 'Ø SENDUNGEN ABH pro Tag', 'Ø SENDUNGEN ZUS pro Tag']
-------------------------------------------------------


In [30]:
# Preprocess Data
postal_codes = df_postalcodes["PLZ (ZAHL)"].tolist()
terminals = df_terminals["Terminal code"].tolist()

# Get capacity
capacity = df_terminals.set_index("Terminal code")["MAX. KAPA SE (# ZUSTELLUNGEN pro Tag)"].to_dict()

# Delivery volume
delivery_volume = df_postalcodes.set_index("PLZ (ZAHL)")["Ø SENDUNGEN ZUS pro Tag"].to_dict()

# Distance dictionary 
distance_dict = {(row["von PLZ (ZAHL)"], row["nach Terminalcode"]): row["Distanz (km)"] for _, row in df_distance.iterrows()}

In [31]:
print("Capacity of each terminals",capacity)
print("\nDelivery volume of each postal codes",delivery_volume)
print("\nPostal codes",postal_codes)

Capacity of each terminals {'AAH': 800, 'AQW': 425, 'BER': 1350, 'BFE': 1390, 'BRE': 1205, 'CBU': 150, 'DUI': 2100, 'EMS': 620, 'ERF': 965, 'FRA': 1940, 'GOH': 530, 'AGB': 900, 'ZEY': 1055, 'HAM': 1800, 'HAJ': 1650, 'QFB': 1355, 'CSO': 660, 'FKB': 850, 'KSF': 1380, 'KHX': 1250, 'ZNV': 1400, 'CGN': 1700, 'ZHZ': 1500, 'ZOL': 1250, 'DMP': 830, 'EUM': 1174, 'NUE': 1555, 'OKZ': 900, 'ZNJ': 1400, 'PVM': 510, 'ZPM': 2200, 'RNM': 1450, 'SCN': 320, 'SEH': 670, 'THF': 720, 'MUC': 2200, 'VIS': 450, 'QUL': 1250, 'ZQV': 1400}

Delivery volume of each postal codes {1067: 13.75, 1069: 3.5, 1097: 2.75, 1099: 5.4375, 1108: 0.75, 1109: 11.875, 1127: 3.3125, 1129: 1.0625, 1139: 10.1875, 1156: 8.3125, 1157: 2.0625, 1159: 3.5625, 1169: 1.75, 1187: 3.1875, 1189: 3.75, 1217: 1.3125, 1219: 4.8125, 1237: 7.4375, 1239: 7.375, 1257: 8.875, 1259: 2.625, 1277: 7.6875, 1279: 2.375, 1307: 2.6875, 1309: 1.875, 1324: 1.0, 1326: 1.0625, 1328: 3.8125, 1445: 11.3125, 1454: 12.0625, 1458: 4.25, 1465: 0.5, 1468: 1.875, 147

In [32]:
sample_plz_list = postal_codes

print("Processing Neighbors for selected PLZs...")

neighbors = {}

# Build neighbors dictionary for selected PLZs
for plz in sample_plz_list:
    # Find all neighbors of 'plz' in the neighbor DataFrame
    neighbor_rows = df_postalcodes_neighbors[df_postalcodes_neighbors['PLZ'] == plz]
    valid_neighbors_in_sample = [
        neighbor for neighbor in neighbor_rows['benachbarte PLZ'] if neighbor in sample_plz_list
    ]
    neighbors[plz] = valid_neighbors_in_sample

# Neighbor constraints
print("Determining neighbor status (z_i) for selected PLZs...")

has_neighbors_z = {}
for plz in sample_plz_list:
    if plz in neighbors and neighbors[plz]:
        has_neighbors_z[plz] = 1
    else:
        has_neighbors_z[plz] = 0

#print("Neighbors..",has_neighbors_z)

Processing Neighbors for selected PLZs...
Determining neighbor status (z_i) for selected PLZs...


In [34]:
# Gurobi Model
model = Model("Capacitated Facility Location- Simpler model")

# Variables
# We are declaring assign as a Binary variable which says whether the postal code is assigned to the terminal or not
assign = model.addVars(postal_codes, terminals, vtype=GRB.BINARY)

x = model.addVars(postal_codes, terminals, vtype=GRB.BINARY)

# Objective Function: Minimize total distance assuming that cost equals distance
model.setObjective(quicksum(assign[p, t] * distance_dict.get((p, t)) for p in postal_codes for t in terminals), GRB.MINIMIZE)

# 1. Constraint- Each postal code is assigned to exactly one terminal
model.addConstrs((quicksum(assign[p, t] for t in terminals) == 1 for p in postal_codes))

# 2. Constraint- Respect Terminal capacity
model.addConstrs((quicksum(assign[p, t] * delivery_volume.get(p, 0) for p in postal_codes) <= capacity[t] for t in terminals))

# 3. Constraint- Island constraint
model.addConstrs((has_neighbors_z[i] * (quicksum(x[k, j] for k in neighbors.get(i, [])) - x[i, j]) >= 0
                  for i in postal_codes for j in terminals))

model.optimize()
print("---------",model.status)

if model.status == GRB.OPTIMAL:
    print("\nOptimal Assignment:")
    for p in postal_codes:
        for t in terminals:
            if assign[p, t].X == 1: #if the postal code is assigned to terminal: print the postal code and corresponding terminal
                print(f"Postal Code {p} assigned to Terminal {t}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.4.0 24E263)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 327359 rows, 638274 columns and 2815371 nonzeros
Model fingerprint: 0x2eba60a1
Variable types: 0 continuous, 638274 integer (638274 binary)
Coefficient statistics:
  Matrix range     [6e-02, 2e+02]
  Objective range  [1e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 3253241.3100
Presolve removed 319289 rows and 325068 columns
Presolve time: 0.64s
Presolved: 8070 rows, 313206 columns, 626412 nonzeros
Found heuristic solution: objective 3195182.5700
Variable types: 0 continuous, 313206 integer (313206 binary)
Performing another presolve...
Presolve time: 0.72s
Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.01s

B